In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn import svm

In [4]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [6]:
from sklearn import preprocessing

In [7]:
from sklearn.model_selection import train_test_split


In [8]:
from sklearn_pandas import DataFrameMapper, cross_val_score

In [9]:
from sklearn.decomposition import PCA

In [172]:
from sklearn.ensemble import GradientBoostingClassifier

In [202]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


In [197]:
from sklearn.model_selection import cross_val_score


In [220]:
from sklearn.ensemble import AdaBoostClassifier


In [229]:
from sklearn.naive_bayes import GaussianNB

In [237]:
from sklearn.ensemble import VotingClassifier


In [122]:
def get_score(y_pred, y_true):
    acc_ = accuracy_score(y_true=y_true,y_pred=y_pred)
    TP = np.sum(((y_pred == 1) & (y_true == 1))) 
    precision = TP / np.sum(y_pred)
    recall = TP / np.sum(y_true)
    print('TP: ',TP,'/', np.sum(y_true), 'all ',np.sum(y_pred), ' accuracy: ',acc_, ' precision: ',precision, ' recall: ',recall, ' F_score: ', 2 * precision * recall / (precision + recall),fbeta_score(y_true=y_true,y_pred=y_pred,beta=1) )

In [123]:
def get_features_middle(data):
    model_sample_strong_feature = data.copy()
    # 将身份信息以及财产信息进行编码
    model_sample_strong_feature['x_022/x_020'] = data['x_022'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_023/x_020'] = data['x_023'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_024/x_020'] = data['x_024'] /  (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_025/x_020'] = data['x_025'] /  (data['x_020']+ 1e-10)
    model_sample_strong_feature['x_026/x_020'] = data['x_026'] /  (data['x_020'] + 1e-10)
    
    # 贷记卡的比例特征
    model_sample_strong_feature['x_028/x_021'] = data['x_028'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_029/x_021'] = data['x_029'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_030/x_021'] = data['x_030'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_031/x_021'] = data['x_031'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_032/x_021'] = data['x_032'] /  (data['x_021'] + 1e-10)
    
    # 银行卡的比例特征
    model_sample_strong_feature['all_cards'] = (data['x_034'] +  data['x_035'] + data['x_036'] + data['x_037'] + data['x_038'] + data['x_039'] + data['x_040']  ).values

    model_sample_strong_feature['x_034/all_cards'] = data['x_034'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_035/all_cards'] = data['x_035'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_036/all_cards'] = data['x_036'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_037/all_cards'] = data['x_037'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_038/all_cards'] = data['x_038'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_039/all_cards'] = data['x_039'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_040/all_cards'] = data['x_040'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
   
    # 标准差还原
    model_sample_strong_feature['x_043/x_044'] = data['x_043'] / (data['x_044'] + 1e-10)
    model_sample_strong_feature['x_046/x_047'] = data['x_046'] / (data['x_047'] + 1e-10)
    model_sample_strong_feature['x_050/x_051'] = data['x_050'] / (data['x_051'] + 1e-10)
    model_sample_strong_feature['x_053/x_054'] = data['x_053'] / (data['x_054'] + 1e-10)
    model_sample_strong_feature['x_057/x_058'] = data['x_057'] / (data['x_058'] + 1e-10)
    model_sample_strong_feature['x_060/x_061'] = data['x_060'] / (data['x_061'] + 1e-10)
    model_sample_strong_feature['x_076/x_077'] = data['x_076'] / (data['x_077'] + 1e-10)
    model_sample_strong_feature['x_079/x_080'] = data['x_079'] / (data['x_080'] + 1e-10)
    model_sample_strong_feature['x_083/x_084'] = data['x_083'] / (data['x_084'] + 1e-10)
    model_sample_strong_feature['x_086/x_087'] = data['x_086'] / (data['x_087'] + 1e-10)
    model_sample_strong_feature['x_090/x_091'] = data['x_090'] / (data['x_091'] + 1e-10)
    model_sample_strong_feature['x_094/x_095'] = data['x_094'] / (data['x_095'] + 1e-10)
    model_sample_strong_feature['x_098/x_099'] = data['x_098'] / (data['x_099'] + 1e-10)
    model_sample_strong_feature['x_123/x_124'] = data['x_123'] / (data['x_124'] + 1e-10)
    model_sample_strong_feature['x_126/x_127'] = data['x_126'] / (data['x_127'] + 1e-10)

    
    # 每张卡（信用or其他）交易金额等；每笔（异地每笔）交易金额等；每笔还款金额等；每笔商旅，保险，家装，金融等的均值特征；每个月的平均交易笔数；其他有意义的均值特征
    
    model_sample_strong_feature['x_045/x_41'] = data['x_045'] / (data['x_041'] + 1e-10)
    model_sample_strong_feature['x_052/x_48'] = data['x_052'] / (data['x_048'] + 1e-10)
    model_sample_strong_feature['x_059/x_55'] = data['x_059'] / (data['x_055'] + 1e-10) 
    model_sample_strong_feature['x_064/x_062'] = data['x_064'] / (data['x_062'] + 1e-10)
    model_sample_strong_feature['x_067/x_065'] = data['x_067'] / (data['x_065'] + 1e-10)  
    model_sample_strong_feature['x_070/x_068'] = data['x_070'] / (data['x_068'] + 1e-10)
    model_sample_strong_feature['x_073/x_071'] = data['x_073'] / (data['x_071'] + 1e-10) 
    model_sample_strong_feature['x_078/x_074'] = data['x_078'] / (data['x_074'] + 1e-10)  
    model_sample_strong_feature['x_085/x_081'] = data['x_085'] / (data['x_081'] + 1e-10) 
    model_sample_strong_feature['x_100/x_101'] = data['x_100'] / (data['x_101'] + 1e-10)
    model_sample_strong_feature['x_102/x_103'] = data['x_102'] / (data['x_103'] + 1e-10) 
    model_sample_strong_feature['x_108/x_105'] = data['x_108'] / (data['x_105'] + 1e-10)
    model_sample_strong_feature['x_104/x_102'] = data['x_104'] / (data['x_102'] + 1e-10) 
    model_sample_strong_feature['x_109/x_110'] = data['x_109'] / (data['x_110'] + 1e-10)
    model_sample_strong_feature['x_111/x_109'] = data['x_111'] / (data['x_109'] + 1e-10) 
    model_sample_strong_feature['x_112/x_113'] = data['x_112'] / (data['x_113'] + 1e-10)
    model_sample_strong_feature['x_114/x_112'] = data['x_114'] / (data['x_112'] + 1e-10) 
    model_sample_strong_feature['x_115/x_116'] = data['x_115'] / (data['x_116'] + 1e-10)
    model_sample_strong_feature['x_117/x_115'] = data['x_117'] / (data['x_115'] + 1e-10)  
    model_sample_strong_feature['x_118/x_119'] = data['x_118'] / (data['x_119'] + 1e-10)
    model_sample_strong_feature['x_120/x_118'] = data['x_120'] / (data['x_118'] + 1e-10) 
    model_sample_strong_feature['x_125/x_121'] = data['x_125'] / (data['x_121'] + 1e-10) 
    model_sample_strong_feature['x_128/x_129'] = data['x_128'] / (data['x_129'] + 1e-10)
    model_sample_strong_feature['x_130/x_128'] = data['x_130'] / (data['x_128'] + 1e-10)

    # 每笔放款金额，每个机构的放款笔数，每个机构的放款金额
    model_sample_strong_feature['x_133/x_134'] = data['x_133'] / (data['x_134'] + 1e-10)
    model_sample_strong_feature['x_133/x_132'] = data['x_133'] / (data['x_132'] + 1e-10)
    model_sample_strong_feature['x_134/x_132'] = data['x_134'] / (data['x_132'] + 1e-10) 
    model_sample_strong_feature['x_138/x_139'] = data['x_138'] / (data['x_139'] + 1e-10)
    model_sample_strong_feature['x_138/x_137'] = data['x_138'] / (data['x_137'] + 1e-10)
    model_sample_strong_feature['x_139/x_137'] = data['x_139'] / (data['x_137'] + 1e-10) 
    model_sample_strong_feature['x_143/x_142'] = data['x_143'] / (data['x_142'] + 1e-10)
    model_sample_strong_feature['x_143/x_144'] = data['x_143'] / (data['x_144'] + 1e-10)
    model_sample_strong_feature['x_144/x_142'] = data['x_144'] / (data['x_142'] + 1e-10)

    # 每个机构的放款均值,失败还款笔数占比
    model_sample_strong_feature['x_151/x_149'] = data['x_151'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_149'] = data['x_152'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_151'] = data['x_152'] / (data['x_151'] + 1e-10)
    model_sample_strong_feature['x_154/x_153'] = data['x_154'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_156/x_153'] = data['x_156'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_157/x_153'] = data['x_157'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_158/x_153'] = data['x_158'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_159/x_153'] = data['x_159'] / (data['x_153'] + 1e-10)  
    model_sample_strong_feature['x_154/x_155'] = data['x_154'] / (data['x_155'] + 1e-10)  

    model_sample_strong_feature['x_164/x_162'] = data['x_164'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_162'] = data['x_165'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_164'] = data['x_165'] / (data['x_164'] + 1e-10)
    model_sample_strong_feature['x_167/x_166'] = data['x_167'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_169/x_166'] = data['x_169'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_170/x_166'] = data['x_170'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_171/x_166'] = data['x_171'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_167/x_168'] = data['x_167'] / (data['x_168'] + 1e-10) 
    model_sample_strong_feature['x_172/x_167'] = data['x_172'] / (data['x_167'] + 1e-10)  

    model_sample_strong_feature['x_177/x_175'] = data['x_177'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_175'] = data['x_178'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_177'] = data['x_178'] / (data['x_177'] + 1e-10)
    model_sample_strong_feature['x_180/x_179'] = data['x_180'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_182/x_179'] = data['x_182'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_183/x_179'] = data['x_183'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_184/x_179'] = data['x_184'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_185/x_180'] = data['x_185'] / (data['x_180'] + 1e-10)
 
    # 90天与30天的申请贷款机构的趋势，180天与90天的申请贷款机构的趋势，180天与30天的申请贷款机构的趋势；90天与30天的成功申请贷款机构的趋势，180天与90天的成功申请贷款机构的趋势，180天；
    # 30天的成功申请贷款机构的趋势；90天与30天的申请贷款笔数的趋势，180天与90天的申请贷款笔数的趋势，180天与30天的申请贷款笔数的趋势90天的申请贷款笔数的趋势
    model_sample_strong_feature['x_189/x_188'] = data['x_189'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_191/x_190'] = data['x_191'] / (data['x_190'] + 1e-10)
    model_sample_strong_feature['x_193/x_192'] = data['x_193'] / (data['x_192'] + 1e-10)
    model_sample_strong_feature['x_195/x_194'] = data['x_195'] / (data['x_194'] + 1e-10)
    model_sample_strong_feature['x_197/x_196'] = data['x_197'] / (data['x_196'] + 1e-10)
    model_sample_strong_feature['x_199/x_198'] = data['x_199'] / (data['x_198'] + 1e-10)
    model_sample_strong_feature['x_196/x_188'] = data['x_196'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_192/x_188'] = data['x_192'] / (data['x_188'] + 1e-10)
                                                        
    return model_sample_strong_feature 

In [159]:
data = pd.read_csv("model.csv")

In [267]:
data = data.dropna(subset=['x_001'])
data[data.iloc[:, 2: ] < 0] = np.nan
X = data.iloc[:, 2:]
training_mean = X.mean()
X = X.fillna(training_mean)
Y = data['y']
X = get_features_middle(X)
X = preprocessing.scale(X)


/home/sdust/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


array([[-0.42854322,  0.19048599,  2.40939352, ..., -0.00796489,
        -0.50683108, -0.40030987],
       [-0.42854322,  0.67200297, -0.41504221, ...,  0.47179292,
        -0.50683108, -0.40030987],
       [-0.42854322,  1.63503691, -0.41504221, ...,  0.47179292,
         0.2657702 ,  0.42492653],
       ...,
       [-0.42854322, -0.13052532, -0.41504221, ..., -0.96748051,
        -0.50683108, -0.40030987],
       [-0.42854322, -1.09355927, -0.41504221, ...,  0.47179292,
         2.58357404,  3.72587216],
       [-0.42854322,  0.51149731, -0.41504221, ...,  0.25783731,
        -0.50683108, -0.40030987]])

In [161]:
pca = PCA(50)

In [162]:
training_X, test_X, training_Y, test_Y = train_test_split(X, Y, test_size=0.3,shuffle = True)

In [163]:
training_X = pca.fit_transform(training_X)

In [242]:
clf = svm.SVC(gamma='auto', probability=True)
clf.fit(training_X, training_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [165]:
from sklearn.tree import DecisionTreeClassifier

In [166]:
regressor = DecisionTreeClassifier(random_state=0)

In [217]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=3, random_state=0).fit(training_X, training_Y)

In [167]:
regressor.fit(training_X, training_Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [221]:
abc = AdaBoostClassifier(n_estimators=100)

In [193]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=None,
    min_samples_split=2, random_state=0)

In [215]:
etc = ExtraTreesClassifier(n_estimators=100, max_depth=None,
    min_samples_split=2, random_state=0)

In [233]:
nb = GaussianNB()

In [234]:
nb.fit(training_X, training_Y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [247]:
eclf = VotingClassifier(estimators=[('gbc', gbc), ('nb', nb), ('clf', clf)], voting='soft', weights=[3,5,1])  # 无权重投票


In [248]:
eclf.fit(training_X, training_Y)

VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=1.0,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                              

In [198]:
cross_val_score(rfc, training_X, training_Y, cv=5)

array([0.8077198 , 0.80200143, 0.80557541, 0.80400572, 0.80543634])

In [194]:
rfc.fit(training_X, training_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [222]:
abc.fit(training_X, training_Y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [212]:
etc.fit(training_X, training_Y)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=30, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [168]:
test_X = pca.transform(test_X)

In [145]:
preY = clf.predict(test_X)

In [169]:
preY1 = regressor.predict(test_X)

In [218]:
preY2 = gbc.predict(test_X)

In [199]:
preY3 = rfc.predict(test_X)

In [213]:
preY4 = etc.predict(test_X)

In [223]:
preY5 = abc.predict(test_X)

In [235]:
preY6 = nb.predict(test_X)

In [249]:
preY7 = eclf.predict(test_X)

In [170]:
pd.Series(preY1).value_counts()

0    2348
1     649
dtype: int64

In [146]:
pd.Series(preY).value_counts()

0    2888
1     109
dtype: int64

In [147]:
get_score(preY, test_Y) # SVM

TP:  49 / 597 all  109  accuracy:  0.7971304637971305  precision:  0.44954128440366975  recall:  0.08207705192629816  F_score:  0.1388101983002833 0.1388101983002833


In [180]:
get_score(preY1, test_Y) # 决策树

TP:  205 / 578 all  649  accuracy:  0.7273940607273941  precision:  0.31587057010785824  recall:  0.3546712802768166  F_score:  0.3341483292583537 0.3341483292583537


In [219]:
get_score(preY2, test_Y) # 梯度下降树

TP:  162 / 578 all  445  accuracy:  0.7667667667667668  precision:  0.36404494382022473  recall:  0.28027681660899656  F_score:  0.31671554252199413 0.31671554252199413


In [201]:
get_score(preY3, test_Y)

TP:  46 / 578 all  79  accuracy:  0.8114781448114782  precision:  0.5822784810126582  recall:  0.07958477508650519  F_score:  0.1400304414003044 0.1400304414003044


In [214]:
get_score(preY4, test_Y)

TP:  29 / 578 all  54  accuracy:  0.8084751418084751  precision:  0.5370370370370371  recall:  0.050173010380622836  F_score:  0.09177215189873418 0.09177215189873418


In [224]:
get_score(preY5, test_Y)

TP:  122 / 578 all  268  accuracy:  0.7991324657991324  precision:  0.4552238805970149  recall:  0.21107266435986158  F_score:  0.2884160756501182 0.2884160756501182


In [236]:
get_score(preY6, test_Y) # 朴素贝叶斯

TP:  185 / 578 all  418  accuracy:  0.7911244577911245  precision:  0.44258373205741625  recall:  0.32006920415224915  F_score:  0.37148594377510036 0.37148594377510036


In [250]:
get_score(preY7, test_Y) # bagging（朴素贝叶斯+决策树+SVM）

TP:  158 / 578 all  341  accuracy:  0.7987987987987988  precision:  0.4633431085043988  recall:  0.27335640138408307  F_score:  0.3438520130576714 0.3438520130576714


In [269]:
test_data = pd.read_csv("test.csv")

In [270]:
test_data

,user_id,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,x_009,...,x_190,x_191,x_192,x_193,x_194,x_195,x_196,x_197,x_198,x_199
0,RaWo4dot7l,0.0,23.0,0,0,0,0,0,0,0,...,10.0,10.0,8.0,8.0,13.0,13.0,11.0,11.0,24.0,24.0
1,CjLuoG%Y93,1.0,25.0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ycwl!Uu%ut,0.0,24.0,1,0,0,0,0,0,0,...,1.0,1.0,4.0,4.0,6.0,6.0,7.0,7.0,11.0,11.0
3,pIfzo8WDm1,0.0,26.0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b%DlFzgWF@,1.0,40.0,0,0,0,0,0,0,0,...,7.0,6.0,7.0,7.0,10.0,8.0,8.0,8.0,14.0,11.0
5,Z2C2o2!Wn6,0.0,27.0,0,0,0,0,0,0,0,...,6.0,6.0,7.0,7.0,13.0,13.0,9.0,9.0,16.0,16.0
6,EEPTUYn0^k,0.0,33.0,1,0,0,1,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,xEN#b4$f4g,0.0,34.0,1,0,0,0,0,0,0,...,0.0,0.0,3.0,3.0,8.0,8.0,3.0,3.0,8.0,8.0
8,jLN3UVi^@4,0.0,30.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9aOJm3JW8g,0.0,38.0,0,0,0,0,0,0,0,...,10.0,10.0,16.0,15.0,29.0,28.0,23.0,22.0,44.0,43.0


In [271]:
test_data = test_data.iloc[:, 1:]
test_data[test_data.iloc[:, 1: ] < 0] = np.nan
test_data = test_data.fillna(training_mean)
test_data = get_features_middle(test_data)
test_data = preprocessing.scale(test_data)

/home/sdust/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [266]:
X

array([[-0.42854322,  0.19048599,  2.40939352, ..., -0.00796489,
        -0.50683108, -0.40030987],
       [-0.42854322,  0.67200297, -0.41504221, ...,  0.47179292,
        -0.50683108, -0.40030987],
       [-0.42854322,  1.63503691, -0.41504221, ...,  0.47179292,
         0.2657702 ,  0.42492653],
       ...,
       [-0.42854322, -0.13052532, -0.41504221, ..., -0.96748051,
        -0.50683108, -0.40030987],
       [-0.42854322, -1.09355927, -0.41504221, ...,  0.47179292,
         2.58357404,  3.72587216],
       [-0.42854322,  0.51149731, -0.41504221, ...,  0.25783731,
        -0.50683108, -0.40030987]])

In [277]:
test_data = pca.transform(test_data)

ValueError: operands could not be broadcast together with shapes (1000,50) (300,) 

In [278]:
res = clf.predict(test_data)

In [279]:
b=np.savetxt('result.csv', res,fmt='%d',delimiter=',')
pd

<module 'pandas' from '/home/sdust/.local/lib/python3.5/site-packages/pandas/__init__.py'>